In [ ]:
import pandas as pd

In [ ]:
import pickle

In [ ]:
import numpy as np
np.random.seed(2018)

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [ ]:
url='https://drive.google.com/file/d/1XY8kQd7kEivHHmU3jdZAZD8s0Anb8XsP/view'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
trainingDf = pd.read_csv(url)
trainingDf.head()

In [ ]:
import gensim
from gensim import corpora
from pprint import pprint

In [ ]:
doc = trainingDf["combine"].values.tolist()


In [ ]:
from gensim.models import Phrases

In [ ]:
bigram = Phrases(doc, min_count=5, threshold=2)

In [ ]:
# Add bigrams to docs (only ones that appear 20 times or more).

for idx in range(len(doc)):
    for token in bigram[doc[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            doc[idx].append(token)
            if idx<1000:
                            

In [ ]:
doc[5]

In [ ]:
text_tokens = [[text for text in doc.split()] for doc in doc]

In [ ]:
dict_LoS = corpora.Dictionary(text_tokens)

In [ ]:
print(dict_LoS)

In [ ]:
try:
    print("Reading corpus from pickle")
    bow_corpus=pickle.load(open('bow_corpusE.pkl', 'rb'))
except FileNotFoundError:
    print("Creating corpus and saving to pickle")
    bow_corpus = [dict_LoS.doc2bow(w.split()) for w in doc]
    pickle.dump(bow_corpus, open('bow_corpusE.pkl', 'wb'))
    pickle.dump(dict_LoS, open('dictionaryE.pkl', 'wb'))

In [ ]:
bow_doc_16 = bow_corpus[1000]

for i in range(len(bow_doc_16)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_16[i][0], 
                                               dict_LoS[bow_doc_16[i][0]], 
                                                bow_doc_16[i][1]))

In [ ]:
try:
    lda_model=pickle.load(open('lda_model20.pkl', 'rb'))
    print("Reading lda_model from pickle")
except FileNotFoundError:
    print("Creating lda_model and saving to pickle")
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dict_LoS, passes=2, workers=2)
    pickle.dump(lda_model,open('lda_model20.pkl','wb'))

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_model, bow_corpus, dict_LoS)

In [ ]:
doc[89]

In [ ]:
lda_model[bow_corpus[89]]

In [ ]:
lda_model.show_topics(20,7)

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
import pyLDAvis.gensim_models

In [ ]:
try:
    lda_display=pickle.load(open('lda_display20.pkl', 'rb'))
    print("Reading lda_display from pickle")
except FileNotFoundError:
    print("Creating lda_display and saving to pickle")
    lda_display = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, 
                                          dict_LoS, mds='mmds')
    pickle.dump(lda_display,open('lda_display20.pkl','wb'))

In [ ]:
pyLDAvis.display(lda_display, template_type='notebook')